In [1]:
import pandas as pd
import re
import time
import numpy as np
import ast
import sys
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
#from sklearn.linear_model import LogisticRegression
#from sklearn.impute import SimpleImputer

If you're checking out the similarity function, use this notebook first. This will get you the non-nested columns and the pitches. To get the timbres, I'd recommend using "nested_column_commandline" and, as its name suggests, running it from the command line. Also, I removed columns that were more than 95% nan before doing the imputing, and split my work over several notebooks. I did all of this because of the fact that this is all pretty memory-intensive, and if I didn't chunk things up one at a time, Python couldn't allocate enough memory for a lot of this. 

So once you've done that, run "nested_column_commandline". After that, run "do_artist_terms". Then run "prep_and_run_cosine_similarity." Those last two will give you PCAs (and in case of timbre/pitch, run some PCAs to make things a lot faster). "prep_and_run_cosine_similarity" also, as the name suggests, contains the similarity function. 

In [3]:
filename = "../EDA/millionsong_subset.csv"
subset = pd.read_csv(filename)

In [4]:
subset.head()

,artist_familiarity,artist_hotttnesss,artist_id,artist_mbid,artist_playmeid,artist_7digitalid,artist_latitude,artist_longitude,artist_location,artist_name,...,sections_confidence,beats_start,beats_confidence,bars_start,bars_confidence,tatums_start,tatums_confidence,artist_mbtags,artist_mbtags_count,year
0,0.581794,0.401998,b'ARD7TVE1187B99BFB1',b'e77e51a5-4761-45b3-9847-2051f811e366',4479,165270,NaN,NaN,b'California - LA',b'Casual',...,"[1.0, 1.0, 0.218, 0.133, 0.384, 0.326, 0.373, ...","[0.58521, 1.19196, 1.78893, 2.37813, 2.94247, ...","[0.834, 0.851, 0.65, 0.635, 0.532, 0.753, 0.62...","[0.58521, 2.94247, 5.14371, 7.74554, 10.36149,...","[0.643, 0.746, 0.722, 0.095, 0.091, 0.362, 0.4...","[0.28519, 0.58521, 0.89422, 1.19196, 1.49119, ...","[0.779, 0.734, 0.674, 0.637, 0.597, 0.532, 0.4...",[],[],0
1,0.630630,0.417500,b'ARMJAGH1187FB546F3',b'1c78ab62-db33-4433-8d0b-7c8dcf1849c2',22066,1998,35.14968,-90.04892,"b'Memphis, TN'",b'The Box Tops',...,"[1.0, 0.451, 0.27, 0.397, 0.225, 0.426, 0.459,...","[0.20627, 0.71054, 1.21836, 1.71841, 2.21729, ...","[1.0, 0.945, 0.714, 0.973, 0.818, 0.974, 0.878...","[0.71054, 2.71502, 4.70861, 6.69288, 8.66941, ...","[0.007, 0.259, 0.172, 0.404, 0.011, 0.016, 0.0...","[0.20627, 0.45904, 0.71054, 0.96834, 1.21836, ...","[0.969, 0.929, 0.897, 0.871, 0.856, 0.845, 0.8...",[b'classic pop and rock'],[1],1969
2,0.487357,0.343428,b'ARKRRTF1187B9984DA',b'7a273984-edd9-4451-9c4d-39b38f05ebcd',13951,290021,NaN,NaN,b'',b'Sonora Santanera',...,"[1.0, 0.121, 0.214, 0.198, 0.66, 0.468, 0.591,...","[0.73152, 1.39732, 2.04852, 2.68691, 3.315, 3....","[0.98, 0.399, 0.185, 0.27, 0.422, 0.0, 0.445, ...","[0.73152, 1.39732, 2.04852, 2.68691, 3.315, 3....","[0.98, 0.399, 0.185, 0.27, 0.422, 0.0, 0.445, ...","[0.42132, 0.73152, 1.06609, 1.39732, 1.72854, ...","[0.482, 0.676, 0.627, 0.549, 0.279, 0.264, 0.2...",[],[],0
3,0.630382,0.454231,b'AR7G5I41187FB4CE6C',b'e188a520-9cb7-4f73-a3d7-2f70c6538e92',12697,19072,NaN,NaN,"b'London, England'",b'Adam Ant',...,"[1.0, 0.086, 0.153, 0.146, 0.088, 0.217, 0.372...","[0.81002, 1.30621, 1.80617, 2.2996, 2.80049, 3...","[0.809, 0.616, 0.789, 0.66, 0.439, 0.758, 0.60...","[1.30621, 3.29887, 5.30252, 7.32327, 9.33775, ...","[0.017, 0.05, 0.014, 0.008, 0.114, 0.019, 0.08...","[0.56254, 0.81002, 1.05749, 1.30621, 1.55494, ...","[0.601, 0.556, 0.523, 0.49, 0.466, 0.44, 0.428...","[b'uk', b'british', b'english']","[1, 1, 1]",1982
4,0.651046,0.401724,b'ARXR32B1187FB57099',b'c6903a2e-063c-4f91-a284-17b8f421be7b',8355,30973,NaN,NaN,b'',b'Gob',...,"[1.0, 0.768, 0.611, 0.388, 0.52, 0.42, 0.499, ...","[0.13576, 0.59914, 1.06368, 1.52591, 1.99045, ...","[0.883, 0.738, 0.484, 0.609, 0.625, 0.719, 0.4...","[1.06368, 2.91491, 4.76729, 6.61852, 8.46978, ...","[0.175, 0.409, 0.639, 0.067, 0.016, 0.066, 0.0...","[0.13576, 0.36918, 0.59914, 0.83141, 1.06368, ...","[1.0, 0.98, 0.932, 0.87, 0.82, 0.793, 0.768, 0...",[],[],2007


In [129]:
use_columns = ['artist_familiarity','artist_hotttnesss','song_hotttnesss', 'key','loudness', 'tempo', 'time_signature','year','artist_location']
use_columns += ['similar_artists'] 

use_subset = subset[use_columns]

In [131]:
mbtag_columns = ['artist_mbtags','artist_terms',]

In [133]:
use_non_nest_list_cols = ['segments_start','segments_loudness_max','segments_loudness_max_time',
                          'segments_loudness_start','sections_start','beats_start','bars_start','tatums_start']
non_nest_list_subset = subset[use_non_nest_list_cols]

In [5]:
possible_nest_list_cols = ['segments_pitches','segments_timbre']
nest_list_subset = subset[possible_nest_list_cols]

In [ ]:
nonnest_len_segstart = non_nest_list_subset['segments_start'].apply(lambda x: ast.literal_eval(x))



In [ ]:
nonnest_len_max = nonnest_len_segstart.apply(lambda x: len(x)).max()

In [58]:
nonnest_segstart_dict = {f'segments_start_{i}': [None for _ in range(nonnest_len_segstart.shape[0])] for i in range(nonnest_len_max)}



(10000,)

In [9]:
out_dict = {}

def create_list_dict(dataframe,col_list,timer=False):
    out_dict = dict()
    if timer: 
        total_time = time.time()
    for col_name in col_list: 
        if timer:
            col_time = time.time()
        print(col_name)
        lit_eval_col = dataframe[col_name].apply(lambda x: ast.literal_eval(x))
        lit_eval_col_max= lit_eval_col.apply(lambda x: len(x)).max()
        print("got through getting lists")
        lit_start_dict = {f'{col_name}_{i}': [None for _ in range(lit_eval_col.shape[0])] for i in range(lit_eval_col_max)}
        for listdex in range(lit_eval_col.shape[0]):
            if listdex % 1000 == 0:
                print(f"Doing row {listdex} out of {lit_eval_col.shape[0]}")
            for dictdex in range(len(lit_eval_col.iloc[listdex])): 
                #print(f'{col_name}_{dictdex}')
                lit_start_dict[f'{col_name}_{dictdex}'][listdex] = lit_eval_col.iloc[listdex][dictdex]
        for i in lit_start_dict.keys(): 
            out_dict[i] = lit_start_dict[i]
        if timer:
            print(f"This column took {time.time() - col_time} seconds")
            print(f"{time.time() - total_time} seconds have passed in total")
        print('-----------------')
    return out_dict

out_dict = create_list_dict(non_nest_list_subset,use_non_nest_list_cols,timer=True)

NameError: name 'non_nest_list_subset' is not defined

In [136]:
#out_df.to_csv('95_percent_full_non_nested_list_columns.csv')

In [19]:
nest_list_subset

,segments_pitches,segments_timbre
0,"[[0.946, 0.684, 0.679, 0.941, 0.744, 0.633, 0....","[[0.0, 171.13, 9.469, -28.48, 57.491, -50.067,..."
1,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[0.0, 171.13, 9.469, -28.48, 57.491, -50.067,..."
2,"[[1.0, 0.911, 0.18, 0.334, 0.327, 0.344, 0.302...","[[0.0, 171.124, 9.459, -28.489, 57.482, -50.06..."
3,"[[0.651, 0.592, 0.647, 0.494, 0.683, 0.919, 1....","[[24.937, 37.465, 177.22, -216.443, 56.3, 202...."
4,"[[1.0, 0.529, 0.407, 0.423, 0.524, 0.509, 0.65...","[[0.089, 169.621, 5.435, -30.061, 54.144, -50...."
...,...,...
9995,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[0.0, 171.13, 9.469, -28.48, 57.491, -50.067,..."
9996,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[0.0, 171.13, 9.469, -28.48, 57.491, -50.067,..."
9997,"[[0.93, 1.0, 0.768, 0.688, 0.758, 0.777, 0.883...","[[0.0, 171.13, 9.469, -28.48, 57.491, -50.067,..."
9998,"[[0.823, 0.272, 0.222, 0.293, 0.241, 0.282, 0....","[[23.471, 185.592, 124.145, -73.753, 68.518, 1..."


In [11]:
def nested_column_to_dict(df, inlist): 
    start = time.time()
    outdict = dict()
    for col in inlist: 
        print("Okay, we're starting in on column " + col)
        coltime = time.time()
        nested_list_column = df[col].apply(lambda x: ast.literal_eval(x))
        nested_list_outer_len = nested_list_column.apply(lambda x: len(x)).max()
        nested_list_inner_len = nested_list_column.apply(lambda x: max([len(i) for i in x])).max()
        lentime = time.time()
        print(f"Getting lengths took {lentime - coltime} seconds. {lentime - start} seconds have passed in total.")
        for outer in range(nested_list_outer_len):
            for inner in range(nested_list_inner_len): 
                outdict[f"{col}_{outer}_{inner}"] = [None for _ in range(df.shape[0])]
        skeletontime = time.time()
        print(f"Setting up the Nones took {skeletontime - lentime} seconds. This column has taken {skeletontime-coltime} seconds. {skeletontime - start} seconds have passed in total")
        for row in range(df.shape[0]): 
            for outer_dex in range(len(nested_list_column.iloc[row])): 
                for inner_dex in range(len(nested_list_column.iloc[row][outer_dex])): 
                    outdict[f"{col}_{outer_dex}_{inner_dex}"][row] = nested_list_column.iloc[row][outer_dex][inner_dex]
            if row % 500 == 0:
                print(f"I am on row {row} out of {df.shape[0]}")
        colendtime = time.time()
        print(f"Doing this column took {colendtime - coltime} seconds. {colendtime-start} seconds have passed in total")
    return outdict


#This only worked on segments_pitches, which is the first column. If you want to do the other one, you'll need to check out nested_columns_may_melt_your_computer.py
expanded_nested_pitches = nested_column_to_dict(nest_list_subset, [list(nest_list_subset.columns)[0]])

Okay, we're starting in on column segments_pitches
Getting lengths took 239.660254240036 seconds. 239.6760959625244 seconds have passed in total.
Setting up the Nones took 25.136075496673584 seconds. This column has taken 264.7963297367096 seconds. 264.812171459198 seconds have passed in total
I am on row 0 out of 10000
I am on row 500 out of 10000
I am on row 1000 out of 10000
I am on row 1500 out of 10000
I am on row 2000 out of 10000
I am on row 2500 out of 10000
I am on row 3000 out of 10000
I am on row 3500 out of 10000
I am on row 4000 out of 10000
I am on row 4500 out of 10000
I am on row 5000 out of 10000
I am on row 5500 out of 10000
I am on row 6000 out of 10000
I am on row 6500 out of 10000
I am on row 7000 out of 10000
I am on row 7500 out of 10000
I am on row 8000 out of 10000
I am on row 8500 out of 10000
I am on row 9000 out of 10000
I am on row 9500 out of 10000
Doing this column took 615.6135740280151 seconds. 615.6294157505035 seconds have passed in total


In [17]:
expanded_nested_pitches= pd.DataFrame(expanded_nested_pitches)

In [43]:
def drop_na_cols(df, threshold = 0.5): 
    df_isnan_sum = df.isna().sum()
    df_isnan_bool = (df_isnan_sum <= df.shape[0] * threshold) == True
    df_isnan_bool = df_isnan_bool[df_isnan_bool].index
    df = df[df_isnan_bool]
    return df

expanded_nested_pitches = drop_na_cols(expanded_nested_pitches)

In [41]:
sys.getsizeof(expanded_nested_pitches)/(1024**2)

702.2096290588379

In [45]:
expanded_nested_pitches.to_csv('pitches_half.csv')

In [155]:
thing = list(expanded_nested_pitches.keys())[-1]
other = list(expanded_nested_pitches.keys())[0]

In [17]:
proper_dict = dict()
for key in expanded_nested_pitches.keys(): 
    if expanded_nested_pitches[key].count(None) >= (len(expanded_nested_pitches[key])*.95):
        proper_dict[key] = expanded_nested_pitches[key]

In [23]:
len(proper_dict.keys())

101028

In [25]:
exp_nest_pitch_df = pd.DataFrame(expanded_nested_pitches)

In [27]:
test_df_size =  sys.getsizeof(exp_nest_pitch_df)
test_df_size/(1024**2)

In [37]:
#exp_nest_pitch_df.to_csv("segments_pitches_flattened.csv")

In [15]:
def listify(instr): 
    outlist = []
    if type(instr) == str:
        rawlist = instr.split(',')
    else: 
        rawlist = instr
    pattern = r"[\'\"](.*)"
    for element in rawlist: 
        red_element = re.findall(pattern,element)
        if len(red_element) != 0: 
            red_element = red_element[0]
            cleaned_element = re.sub(r'[\"\'\]\[]','',red_element)
            outlist.append(cleaned_element)
    return outlist

def collect_list(use_col):
    out_dict = dict()
    for element in use_col: 
        listed_el = listify(element)
        for sub_element in listed_el: 
            if sub_element not in out_dict.keys(): 
                out_dict[sub_element] = 1
            else: 
                out_dict[sub_element] += 1
    return out_dict

def sort_genre(in_dict): 
    uselist = [(i,in_dict[i]) for i in in_dict]
    uselist = sorted(uselist, key = lambda x: -x[1])
    return uselist

artist_term_dict = collect_list(subset['artist_terms'])
artist_term_sorted = sort_genre(artist_term_dict)

artist_mbtags_dict = collect_list(subset['artist_mbtags'])
artist_mbtags_sorted = sort_genre(artist_mbtags_dict)

In [121]:
#artist_term_sorted

In [31]:
#test = pd.read_csv('segments_timbre_unpacked_95_percent_nonnan.csv')